In [1]:
import os
%pwd

'c:\\Users\\Vallee\\Stack\\__DSProjects__\\article_research_bot\\notebooks'

In [2]:
os.chdir("../")
%pwd

'c:\\Users\\Vallee\\Stack\\__DSProjects__\\article_research_bot'

In [ ]:
#Install Packages
# !pip install faiss-cpu
# !pip install sentence-transformers

In [3]:
# import necessary libraries
import pandas as pd
pd.set_option('display.max_colwidth', 100)

In [4]:
df = pd.read_csv("notebooks/sample_text.csv")
df.shape

(8, 2)

In [5]:
df

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


### Step 1 : Create source embeddings for the text column

In [6]:
from sentence_transformers import SentenceTransformer

c:\Users\Vallee\anaconda3\envs\articleenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
# Hugging Face
encoder = SentenceTransformer("all-mpnet-base-v2")
vectors = encoder.encode(df.text)

c:\Users\Vallee\anaconda3\envs\articleenv\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Vallee\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [8]:
vectors.shape

(8, 768)

In [9]:
vectors

array([[-0.00247386,  0.03626738, -0.05290457, ..., -0.09152357,
        -0.03970006, -0.04330486],
       [-0.03357265,  0.0098052 , -0.03250131, ..., -0.05165471,
         0.02245886, -0.0315618 ],
       [-0.01865333, -0.04051305, -0.01235392, ...,  0.00610591,
        -0.07179645,  0.02773852],
       ...,
       [-0.00066453,  0.04252131, -0.05645506, ...,  0.0131547 ,
        -0.03183563, -0.04357663],
       [-0.03317152,  0.03252462, -0.02484841, ...,  0.01174419,
         0.0574712 ,  0.00571026],
       [-0.00166399,  0.00413823, -0.04597077, ...,  0.02008529,
         0.05656242, -0.00161591]], dtype=float32)

In [10]:
dim = vectors.shape[1]
dim

768

### Step 2 : Build a FAISS Index for vectors

In [11]:
import faiss

# IndexFlatL2 uses the euclidean/l2 distance, helps with fast searching
index = faiss.IndexFlatL2(dim)

### Step 3 : Normalize the source vectors (as we are using L2 distance to measure similarity) and add to the index

In [12]:
index.add(vectors)

In [13]:
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x0000022C2656BAE0> >

### Step 4 : Encode search text using same encorder and normalize the output vector

In [26]:
# search_query = "I want to buy a polo t-shirt"
search_query = "looking for places to visit during the holidays"
# search_query = "An apple a day keeps the doctor away"
vec = encoder.encode(search_query)
vec.shape

(768,)

In [27]:
import numpy as np

# converting to 2D array cos the search vector expects so
svec = np.array(vec).reshape(1,-1)
svec.shape

(1, 768)

In [28]:
svec

array([[-5.69070578e-02, -7.85626005e-03, -5.97458743e-02,
         1.70310866e-02, -1.56358052e-02, -1.14044799e-02,
         1.34391142e-02,  1.58604365e-02, -5.63505339e-03,
        -3.65664065e-02, -2.79401038e-02,  9.15883668e-03,
         6.13885969e-02, -8.18535313e-02, -4.09648521e-03,
        -4.49632891e-02,  1.87594257e-02, -2.79248189e-02,
        -6.87939376e-02, -3.13307680e-02, -3.87345925e-02,
         2.53378171e-02, -2.10150098e-03,  1.09867593e-02,
         6.78736791e-02, -1.60868187e-02, -5.97489513e-02,
        -1.26970289e-02,  3.33298147e-02,  4.20751609e-02,
        -3.19535807e-02, -2.43929178e-02,  6.78693643e-03,
         2.02426147e-02,  1.25888391e-06,  4.73243520e-02,
         2.17979308e-02, -3.91112417e-02,  3.06510758e-02,
         9.65031516e-03, -3.92406806e-02, -4.86151278e-02,
         4.31999704e-03,  1.74005963e-02,  2.01661643e-02,
        -7.81874806e-02, -3.52568156e-03, -2.18736771e-02,
        -3.01542152e-02,  1.45583889e-02,  4.68927249e-0

In [67]:
# faiss.normalize_L2(svec)

### Step 5: Search for similar vector in the FAISS index created

In [29]:
index.search(svec, k=2)

(array([[0.92737234, 1.1601744 ]], dtype=float32),
 array([[6, 7]], dtype=int64))

In [30]:
distances, I = index.search(svec, k=2)
distances

array([[0.92737234, 1.1601744 ]], dtype=float32)

In [31]:
I

array([[6, 7]], dtype=int64)

In [32]:
# df.loc[[3,2]]
df.loc[I[0]]

,text,category
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


In [33]:
I.tolist()

[[6, 7]]

In [34]:
row_indices = I.tolist()[0]
row_indices

[6, 7]

In [35]:
df.loc[row_indices]

,text,category
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


In [36]:
search_query

'looking for places to visit during the holidays'

You can see that the two results from the dataframe are similar to a search_query, which is a semantic search